# Imports

In [2]:
import numpy as np
import pandas as pd
from pandas import DataFrame as dataframe
import matplotlib.pyplot as plt
# import seaborn as sns
import os
from os.path import join
# from tqdm import tqdm


In [3]:
print(os.getcwd())

base_dir = os.getcwd()
paths={
    'raw': join(base_dir, 'data'),
    'processed': join(base_dir, 'processed'),
}

D:\mh100\MH-100K-dataset


In [4]:
os.listdir(paths['processed'])

['motodroid-v2-features-all.csv',
 'motodroid-v2-features-classes.csv',
 'motodroid-v2-vt_labels.csv',
 'motodroid-v2.npy',
 'motodroid-v2_labels.csv']

In [5]:
files = dataframe(os.listdir(paths['processed']), columns=['filename'])
files

,filename
0,motodroid-v2-features-all.csv
1,motodroid-v2-features-classes.csv
2,motodroid-v2-vt_labels.csv
3,motodroid-v2.npy
4,motodroid-v2_labels.csv


# MH100 dataset

* API Call:  features
* Permission:  features
* Intent:  features

* Labels: (CLASS, vt_detection)


In [ ]:
import tensorflow as tf

In [ ]:
print(base_dir)

print(os.listdir(base_dir))
path_dataset = join(base_dir, 'datasets1', 'motodroid-v2')

os.listdir(path_dataset)

In [ ]:
feat_path = join(path_dataset, 'motodroid-v2-features-all.csv')
feat_path

In [ ]:

features = {
    # 'intents': pd.read_csv(join(base_dir, 'features', 'features-intents.csv'), header=0),
    # 'api': pd.read_csv(join(base_dir, 'features', 'features-api.csv')),
    # 'permissions': pd.read_csv(join(base_dir, 'features', 'features-permissions.csv')),
    'classes': pd.read_csv(join(path_dataset, 'motodroid-v2-features-classes.csv')),
    'all': pd.read_csv(join(path_dataset, 'motodroid-v2-features-all.csv')),
    # 'int8': pd.read_csv(join(base_dir, 'features', 'features-int8.csv')),
}

In [ ]:
features['all']

## Load Motodroid RAW dataset

In [ ]:
files = dataframe(os.listdir(join(paths['datasets'], 'motodroid-v2')), columns=['filename'])
files

In [ ]:
# path = 'C:\\motodroid\\MotoDroid_VT_AZ_100K_int.csv'
dataset_name = files['filename'].iloc[2]
print(dataset_name)

path = join(paths['datasets'], 'motodroid-v2', dataset_name)
print(path)

In [ ]:
# data = pd.read_csv(
#     path, 
#     usecols=features['all']['index'].values, 
#     dtype=np.int8,
#     # nrows=1000
# )

# # data.head()

# print(data.info())

# # DATAFRAM WITH CHUNK
# count = 0
# for df in tqdm(pd.read_csv(
#     path,  
#     chunksize=5000, 
#     usecols=features['all']['index'].values, 
#     dtype=np.int8)):
#     # print(df.shape)
#     np.save(join(paths['data'], str(count)+'_motodroid.npy'), df.values)
#     count = count +1

In [ ]:
dataframe(data.columns, columns=['features']).to_csv('motodroid-v2-features.csv')

## Process dataset

In [ ]:
def process_motodroid(path, chunksize, col_index, dtype, path_save):
    # DATAFRAM WITH CHUNK
    count = 0
    for df in tqdm(pd.read_csv(
        path,  
        chunksize=chunksize, 
        usecols=col_index, #features['all']['index'].values, 
        dtype=dtype)):
        # print(df.shape)
        np.save(join(path_save, str(count)+'_motodroid.npy'), df.values)
        count = count +1
    
    return 

def load_motodroid_chunks(path, base_name='_motodroid', path_save=[]):
    
    dataset = []
    
    files = os.listdir(paths['data'])
    print('Reading ', len(files), ' files...')
    for i in tqdm( range(0, len(files))):

        name = str(i)+base_name+'.npy'
        path_name = join(paths['data'], name)
        # print(path_name)

        data__ = np.load(path_name)
        dataset.append(data__)
    dataset = np.concatenate(dataset)
    
    print(dataset.shape)
    
    if path_save:
        np.save(join(path_save, 'motodroid-dataset.npy'), dataset)
    return dataset

In [ ]:
dataset = load_motodroid_chunks(path=paths['data'], 
                                base_name='_motodroid',
                                path_save=paths['processed'])

### Process labels

In [ ]:
def process_motodroid_labels(path, usecols, path_save=[]):
    df_labels = pd.read_csv(
        path,
        usecols=usecols
    )
    
    if path_save:
        np.save(join(path_save, 'motodroid-dataset_labels.npy'), df_labels)
        df_labels.to_csv(path_save, 'motodroid-dataset_labels.csv')
    return df_labels



def create_class(df, threshold):
    return [1 if i>=threshold else 0 for i in df ]
    

In [ ]:
df_labels = process_motodroid_labels(
    path=path, 
    usecols=features['classes']['index'].values, 
    # path_save=paths['processed']
)
df_labels.head()

In [ ]:
## https://www.tensorflow.org/api_docs/python/tf/data/experimental/make_csv_dataset

In [ ]:
df_labels['SHA256'].iloc[2]

In [ ]:
df_labels['vt_detection']

In [ ]:
np.unique(df_labels['vt_detection'])

In [ ]:
class__ = create_class(df=df_labels['vt_detection'] , threshold=2)

# class__

In [ ]:
df = dataframe(df_labels['CLASS'].values, columns=['1-class'])
df

In [ ]:
for vt in np.unique(df_labels['vt_detection']):
    # print(vt)
    if vt > 1:
        class__ = create_class(df=df_labels['vt_detection'] , threshold=vt)
        class_name = str(vt)+'-class'
        df[class_name] = class__


In [ ]:
df

In [ ]:
for c in df.columns:
    values, counts = np.unique(df[c], return_counts=True)
    print(c, counts)


In [ ]:
df.to_csv(join(paths['processed'], 'motodroid-v2-vt_labels.csv'))

In [ ]:
# dataset = tf.data.experimental.make_csv_dataset(filename__[0], 
#                                                 batch_size=10,
#                                                 num_epochs=2,
#                                                 shuffle=True
#                                                )


## Load Processed data

In [ ]:
def select_columns__(arr, col_indices):
    data = arr[:, col_indices]
    return data

In [ ]:
select_columns =  pd.read_csv(join(paths['processed'], 'features-non-api.csv'))
select_columns

In [ ]:
df_labels = dataframe(df_labels, columns=select_columns['features'])


In [ ]:
if not select_columns.empty:
    print('Selected columns')
    dataset = np.load(join(paths['processed'], 'motodroid.npy'))
    dataset = select_columns__(arr=dataset, col_indices=select_columns['index'].values)
    df_labels = np.load(join(paths['processed'], 'motodroid_labels.npy'), allow_pickle=True)
    column_names = pd.read_csv(join(paths['processed'], 'label_names.csv'))
    df_labels = dataframe(df_labels, columns=column_names['features'])
    
else:
    dataset = np.load(join(paths['processed'], 'motodroid.npy'))
    df_labels = np.load(join(paths['processed'], 'motodroid_labels.npy'))
    column_names = pd.read_csv(join(paths['processed'], 'features-int8.csv'))

In [ ]:
print(dataset.shape, df_labels.shape)

In [ ]:
dataset.shape

In [ ]:
df_labels['CLASS']

# Load Motodroid dataset

In [ ]:
os.listdir(paths['processed'])

In [ ]:
def load_motodroid(path, dataset_name):
    dataset = np.load(join(path, dataset_name+'.npy'))
    # df_labels = np.load(join(path, dataset_name+'_labels.npy'))
    data_labels = pd.read_csv(join(path, dataset_name+'_labels.csv'))
    vt_labels = pd.read_csv(join(path, dataset_name+'-vt_labels.csv'))
    vt_labels.drop(columns=['index'], inplace=True)
    classes_names = ['authentic', 'malware']
    return dataset, data_labels, vt_labels, classes_names

def plot_grid(vt_labels, path_save= [], name='plot grid', figsize=(18, 20)):
    fig = plt.figure(figsize=figsize)
    for i, vt in zip (range(0, vt_labels.shape[0]), vt_labels.columns):
        # print(i, idx)
        ax = plt.subplot(7, 6, i + 1)
        # plt.plot(X[idx_sample])
        unique_tr, counts_tr = np.unique(vt_labels[vt], return_counts=True)

        plt.pie(counts_tr, autopct=lambda p : '{:.1f}% ({:,.0f})'.format(p, p * sum(counts_tr)/100), radius=1.5)

        plt.title(vt)
        # plt.ylim(ylim)
        plt.axis("off")
    plt.tight_layout(pad=0.4, w_pad=0.5, h_pad=1.0)
    plt.show()
    if path_save:
        fig.savefig(os.path.join(path_save, name+'.png'), format='png', bbox_inches='tight')
    return 

def class_distribution(y_data, classes):
        unique_tr, counts_tr = np.unique(y_data, return_counts=True)
        dict(zip(unique_tr, counts_tr))

        pie = plt.pie(counts_tr, autopct=lambda p : '{:.1f}% ({:,.0f})'.format(p, p * sum(counts_tr)/100), radius=1)
        plt.legend(pie[0], shadow=True, loc="lower right", 
                                  bbox_transform=plt.gcf().transFigure)
        plt.show()

In [ ]:
data, data_labels, vt_labels, classes_names = load_motodroid(
    path=paths['processed'], 
    dataset_name='motodroid-v2'
)

In [ ]:
print(data.shape)

